In [1]:
# Install libraries
!pip install transformers
!pip install torch
!pip install nltk
!pip install gensim
!pip install arabic_reshaper
!pip install bidi.algorithm


ERROR: Could not find a version that satisfies the requirement bidi.algorithm (from versions: none)
ERROR: No matching distribution found for bidi.algorithm


Importing Necessary libraries

In [2]:
import gensim.downloader as api
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

Chargement de model pré-entrainé CBOW

In [3]:
model_w2v = api.load("word2vec-google-news-300")  # CBOW pré-entrainé


[==================================================] 100.0% 1662.8/1662.8MB downloaded


Charger un modèle BERT pour le contexte lexical


In [4]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model_bert = BertModel.from_pretrained("bert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Fonctions de détection des erreurs typographiques

In [5]:
def detect_typo(word, context_words):
    """
    Détecte une erreur typographique dans le mot donné, en calculant la distance
    sémantique entre le mot et le contexte.
    """
    if word in model_w2v.key_to_index:
        word_vector = model_w2v[word]
        context_vectors = [model_w2v[w] for w in context_words if w in model_w2v.key_to_index]
        if context_vectors:
            context_mean = np.mean(context_vectors, axis=0)
            similarity = cosine_similarity([word_vector], [context_mean])[0][0]
            if similarity < 0.5:  # Ajuster ce seuil selon le besoin
                return True, similarity
    return False, None

In [6]:
from sklearn.metrics.pairwise import cosine_similarity
import torch

def detect_semantic_error(sentence):
    """
    Detects semantic errors in a sentence using BERT embeddings.
    """
    inputs = tokenizer(sentence, return_tensors="pt")
    outputs = model_bert(**inputs)
    sentence_embedding = outputs.last_hidden_state.mean(dim=1)
    words = sentence.split()

    for word in words:
        inputs_word = tokenizer(word, return_tensors="pt")
        outputs_word = model_bert(**inputs_word)
        word_embedding = outputs_word.last_hidden_state.mean(dim=1)

        similarity = cosine_similarity(sentence_embedding.detach().numpy(), word_embedding.detach().numpy())[0][0]

        if similarity < 0.5:  # Adjust threshold as necessary for semantic detection
            return word, similarity

    return None, None


mANUEL VALIDATION

In [7]:
def manual_validation(sentence, predicted_error):
    """
    Cette fonction vérifie si le modèle a détecté correctement les erreurs.
    """
    print(f"Phrase analysée : {sentence}")
    if predicted_error:
        print(f"Erreur détectée sur le mot : {predicted_error[0]} avec une similarité de {predicted_error[1]}")
    else:
        print("Aucune erreur détectée par le modèle.")


Testing with different input sentances

In [8]:
sentences = [
    "Le garçon mange un râteau",
    "Elle a reçu un cadeau de sa tante",
    "Le soleil brille tous les jours"
]

for sentence in sentences:
    print("\nAnalyse de la phrase:", sentence)
    words = sentence.split()
    for word in words:
        typo_detected, typo_similarity = detect_typo(word, [w for w in words if w != word])
        if typo_detected:
            print(f"Erreur typographique potentielle dans le mot '{word}' (similarité: {typo_similarity})")

    semantic_error = detect_semantic_error(sentence)
    if semantic_error[0]:
        print(f"Erreur sémantique potentielle sur le mot '{semantic_error[0]}' (similarité: {semantic_error[1]})")


Analyse de la phrase: Le garçon mange un râteau
Erreur typographique potentielle dans le mot 'Le' (similarité: 0.14314872026443481)
Erreur typographique potentielle dans le mot 'mange' (similarité: 0.11718770861625671)
Erreur typographique potentielle dans le mot 'un' (similarité: 0.2300054132938385)
Erreur sémantique potentielle sur le mot 'Le' (similarité: 0.4041855037212372)

Analyse de la phrase: Elle a reçu un cadeau de sa tante
Erreur typographique potentielle dans le mot 'Elle' (similarité: 0.1807975471019745)
Erreur typographique potentielle dans le mot 'reçu' (similarité: 0.4750600755214691)
Erreur typographique potentielle dans le mot 'un' (similarité: 0.35263147950172424)
Erreur typographique potentielle dans le mot 'de' (similarité: 0.4938667118549347)
Erreur typographique potentielle dans le mot 'sa' (similarité: 0.4255477488040924)
Erreur typographique potentielle dans le mot 'tante' (similarité: 0.43297910690307617)
Erreur sémantique potentielle sur le mot 'Elle' (simil